<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Azure Machine Learning Studio engine

This notebook shows how to log the payload for the model deployed on custom model serving engine using Watson OpenScale python sdk.

Contents
- [1. Setup](#setup)
- [2. Binding machine learning engine](#binding)
- [3. Subscriptions](#subscription)
- [4. Scoring and payload logging](#scoring)
- [5. Feedback logging](#feedback)
- [6. Data Mart](#datamart)

<a id="setup"></a>
## 1. Setup

### 1.0 Sample model creation using [Azure Machine Learning Studio](https://studio.azureml.net)

- Download training data set from [here](https://github.com/pmservice/wml-sample-models/raw/master/spark/product-line-prediction/data/GoSales_Tx.csv)
- Copy [credit risk experiment](https://gallery.cortanaintelligence.com/Experiment/German-credit-risk-created-on-1-9-2019) from Azure ML Studio Gallery 
- Run experiment to train a model
- Create (deploy) web service (new)

**NOTE:** Classic web services are not supported.

### 1.1 Installation and authentication

In [1]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1

Import and initiate.

In [3]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.supporting_classes import PayloadRecord
from ibm_ai_openscale.supporting_classes.enums import InputDataType, ProblemType
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *

#### ACTION: Get Watson OpenScale `instance_guid` and `apikey`

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using bluemix console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

How to get your OpenScale instance GUID

- if your resource group is different than `default` switch to resource group containing OpenScale instance
```
bx target -g <myResourceGroup>
```
- get details of the instance
```
bx resource service-instance 'AI-OpenScale-instance_name'
```

#### Let's define some constants required to set up data mart:

- AIOS_CREDENTIALS
- POSTGRES_CREDENTIALS
- SCHEMA_NAME

In [4]:
AIOS_CREDENTIALS = {
  "url": "https://api.aiopenscale.cloud.ibm.com",
  "instance_guid": "***",
  "apikey": "***"
}

In [5]:
# The code was removed by Watson Studio for sharing.

In [6]:
POSTGRES_CREDENTIALS = {
    "db_type": "postgresql",
    "uri_cli_1": "xxx",
    "maps": [],
    "instance_administration_api": {
        "instance_id": "xxx",
        "root": "xxx",
        "deployment_id": "xxx"
    },
    "name": "xxx",
    "uri_cli": "xxx",
    "uri_direct_1": "xxx",
    "ca_certificate_base64": "xxx",
    "deployment_id": "xxx",
    "uri": "xxx"
}

In [7]:
# The code was removed by Watson Studio for sharing.

In [8]:
SCHEMA_NAME = 'data_mart_for_azure'

Create schema for data mart.

In [9]:
create_postgres_schema(postgres_credentials=POSTGRES_CREDENTIALS, schema_name=SCHEMA_NAME)

In [10]:
client = APIClient(AIOS_CREDENTIALS)

In [11]:
client.version

'2.0.18'

### 1.2 DataMart setup

In [14]:
client.data_mart.setup(db_credentials=POSTGRES_CREDENTIALS, schema=SCHEMA_NAME)

In [15]:
data_mart_details = client.data_mart.get_details()

<a id="binding"></a>
## 2. Bind machine learning engines

### 2.1 Bind  `Azure` machine learning engine

Provide credentials using following fields:
- `client_id`
- `client_secret`
- `subscription_id`
- `tenant`

In [16]:
AZURE_ENGINE_CREDENTIALS = {
    "client_id": "***",
    "client_secret": "***",
    "subscription_id": "***",
    "tenant": "***"
}

In [17]:
# The code was removed by Watson Studio for sharing.

In [18]:
binding_uid = client.data_mart.bindings.add('My Azure ML Studio engine', AzureMachineLearningInstance(AZURE_ENGINE_CREDENTIALS))

In [19]:
bindings_details = client.data_mart.bindings.get_details()

In [20]:
client.data_mart.bindings.list()

91d749e3-8551-45da-84fc-385e7ed2ba26,My Azure ML Studio engine,azure_machine_learning,2019-02-07T12:44:20.125Z


<a id="subsciption"></a>
## 3. Subscriptions

### 3.1 Add subscriptions

List available deployments.

**Note:** Depending on number of assets it may take some time.

In [21]:
client.data_mart.bindings.list_assets()

eb4c24fd0e14d53007a50aef50dbb501,IncomeExperiment.2019.2.6.23.10.17.731,2019-02-06T23:11:50.528911Z,model,,91d749e3-8551-45da-84fc-385e7ed2ba26,False
4fff12e2b55fe1038b2e04d98be759c5,FastpathGermanCreditRisk.2019.2.1.17.17.21.412,2019-02-01T17:17:55.575148Z,model,,91d749e3-8551-45da-84fc-385e7ed2ba26,False
986fd3e779b52d0e23a2bde5b6da996c,ScottdaAzureML12.2019.1.29.21.29.50.528,2019-01-29T21:30:38.2982264Z,model,,91d749e3-8551-45da-84fc-385e7ed2ba26,False
9b6abb9a934cecf33a465f07d73aabe6,MultiClass-ProductLineTestAutomationNC,2019-01-21T06:50:06.8313304Z,model,,91d749e3-8551-45da-84fc-385e7ed2ba26,False
33ee79fb8e64881e9182dd4fd3475586,ClaimInsuranceRegressionTestAutomationCat,2019-01-21T06:46:49.6811946Z,model,,91d749e3-8551-45da-84fc-385e7ed2ba26,False
97521fdfaf0830239fadfbac0d5d0091,ClaimInsuranceRegressionTestAutomationNC,2019-01-21T06:41:04.8380274Z,model,,91d749e3-8551-45da-84fc-385e7ed2ba26,False
a7474537b6166d3cc01481b647423769,Germancreditrisk.2019.1.17.14.9.9.994,2019-01-17T14:09:46.5811815Z,model,,91d749e3-8551-45da-84fc-385e7ed2ba26,False
085460ef94636166aea5800e9ea26168,GermanCreditRisk.2019.1.9.10.41.58.611,2019-01-09T10:42:59.7933412Z,model,,91d749e3-8551-45da-84fc-385e7ed2ba26,False
77919091f862ab6a128edc60ccafc064,ASHDrugModelPred.2019.1.8.9.48.15.590,2019-01-08T09:48:33.9184573Z,model,,91d749e3-8551-45da-84fc-385e7ed2ba26,False
920beeba5513b8059799900ecfcbdeca,MultiClass-ProductLineTestAutomation,2019-01-05T05:55:41.6061263Z,model,,91d749e3-8551-45da-84fc-385e7ed2ba26,False
56f60f0e2f997ed58d851e5a063eb554,ClaimInsuranceRegressionTestAutomation,2019-01-04T10:43:02.5032489Z,model,,91d749e3-8551-45da-84fc-385e7ed2ba26,False


**Action:** Assign your credit risk source_uid to `source_uid` variable below.

In [22]:
source_uid = '085460ef94636166aea5800e9ea26168'

In [23]:
subscription = client.data_mart.subscriptions.add(
    AzureMachineLearningAsset(source_uid=source_uid,
                                      binding_uid=binding_uid,
                                      input_data_type=InputDataType.STRUCTURED,
                                      problem_type=ProblemType.BINARY_CLASSIFICATION,
                                      label_column='Risk',
                                      prediction_column='Scored Labels',
                                      probability_column='Scored Probabilities'))

#### Get subscriptions list

In [24]:
subscriptions = client.data_mart.subscriptions.get_details()

In [25]:
subscriptions_uids = client.data_mart.subscriptions.get_uids()
print(subscriptions_uids)

['085460ef94636166aea5800e9ea26168']


#### List subscriptions

In [26]:
client.data_mart.subscriptions.list()

085460ef94636166aea5800e9ea26168,GermanCreditRisk.2019.1.9.10.41.58.611,model,91d749e3-8551-45da-84fc-385e7ed2ba26,2019-02-07T12:46:42.998Z


<a id="scoring"></a>
## 4. Scoring and payload logging

### 4.1 Score the product line model and measure response time

In [27]:
import requests
import time
import json

subscription_details = subscription.get_details()
scoring_url = subscription_details['entity']['deployments'][0]['scoring_endpoint']['url']

data = {
            "Inputs": {
                "input1":
                    [
                        {
                            'CheckingStatus': "0_to_200",
                            'LoanDuration': 31,
                            'CreditHistory': "credits_paid_to_date",
                            'LoanPurpose': "other",
                            'LoanAmount': 1889,
                            'ExistingSavings': "100_to_500",
                            'EmploymentDuration': "less_1",
                            'InstallmentPercent': 3,
                            'Sex': "female",
                            'OthersOnLoan': "none",
                            'CurrentResidenceDuration': 3,
                            'OwnsProperty': "savings_insurance",
                            'Age': 32,
                            'InstallmentPlans': "none",
                            'Housing': "own",
                            'ExistingCreditsCount': 1,
                            'Job': "skilled",
                            'Dependents': 1,
                            'Telephone': "none",
                            'ForeignWorker': "yes",
                        }
                    ],
            },
            "GlobalParameters": {
            }
        }

body = str.encode(json.dumps(data))

token = subscription_details['entity']['deployments'][0]['scoring_endpoint']['credentials']['token']
headers = subscription_details['entity']['deployments'][0]['scoring_endpoint']['request_headers']
headers['Authorization'] = ('Bearer ' + token)

start_time = time.time()
response = requests.post(url=scoring_url, data=body, headers=headers)
response_time = int(time.time() - start_time)*1000
result = response.json()

print(json.dumps(result, indent=2))

{
  "Results": {
    "output1": [
      {
        "Job": "skilled",
        "CheckingStatus": "0_to_200",
        "CreditHistory": "credits_paid_to_date",
        "Scored Probabilities": "0.0140035906806588",
        "InstallmentPercent": "3",
        "LoanAmount": "1889",
        "OwnsProperty": "savings_insurance",
        "Housing": "own",
        "Sex": "female",
        "LoanPurpose": "other",
        "InstallmentPlans": "none",
        "CurrentResidenceDuration": "3",
        "ExistingSavings": "100_to_500",
        "OthersOnLoan": "none",
        "Dependents": "1",
        "ExistingCreditsCount": "1",
        "Scored Labels": "No Risk",
        "LoanDuration": "31",
        "EmploymentDuration": "less_1",
        "Telephone": "none",
        "Age": "32",
        "ForeignWorker": "yes"
      }
    ]
  }
}


### 4.2 Store the request and response in payload logging table

**Hint:** You can embed payload logging code into your custom deployment so it is logged automatically each time you score the model.

In [28]:
records_list = []

for i in range(1, 10):
    records_list.append(PayloadRecord(request=data, response=result, response_time=response_time))

subscription.payload_logging.store(records=records_list)

<a id="feedback"></a>
## 5. Feedback logging & quality (accuracy) monitoring

### Enable quality monitoring

You need to provide the monitoring `threshold` and `min_records` (minimal number of feedback records).

In [29]:
subscription.quality_monitoring.enable(threshold=0.7, min_records=10)

### Feedback records logging

Feedback records are used to evaluate your model. The predicted values are compared to real values (feedback records).

You can check the schema of feedback table using below method.

In [30]:
subscription.feedback_logging.print_table_schema()

LoanAmount,integer,True
InstallmentPercent,integer,True
ExistingSavings,string,True
CheckingStatus,string,True
LoanPurpose,string,True
Job,string,True
InstallmentPlans,string,True
OthersOnLoan,string,True
LoanDuration,integer,True
Age,integer,True
CreditHistory,string,True


The feedback records can be send to feedback table using below code.

In [31]:
feedback_records = []
fields = ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'Risk']
record = ["0_to_200", 18, "outstanding_credit", "car_new", 884, "less_100", "greater_7", 4, "male", "none", 4, "car_other", 36, "bank", "own", 1, "skilled", 2, "yes", "yes", "Risk"]

for i in range(0, 20):
    feedback_records.append(record)


subscription.feedback_logging.store(feedback_data=feedback_records, fields=fields)

### Run quality monitoring on demand

By default, quality monitoring is run on hourly schedule. You can also trigger it on demand using below code.

In [46]:
run_details = subscription.quality_monitoring.run(background_mode=False)




 Waiting for end of quality monitoring run 87a69dfe-3830-4e47-9372-1c48992ae57c 




initializing
completed

---------------------------
 Successfully finished run 
---------------------------




### Show the quality metrics

In [47]:
subscription.quality_monitoring.show_table()

2019-01-30 11:13:11.803000+00:00,1.0,0.7,dd096f50-15eb-4d22-bc5f-39aff54acc98,085460ef94636166aea5800e9ea26168,563f01d37f720857b95c557dc76176ad,Accuracy_evaluation_63a87c06-3f04-4174-8bf8-77de76c55193,
2019-01-30 11:13:29.674000+00:00,1.0,0.7,dd096f50-15eb-4d22-bc5f-39aff54acc98,085460ef94636166aea5800e9ea26168,563f01d37f720857b95c557dc76176ad,Accuracy_evaluation_87a69dfe-3830-4e47-9372-1c48992ae57c,


Get all calculated metrics.

In [48]:
deployment_uids = subscription.get_deployment_uids()

In [49]:
subscription.quality_monitoring.get_metrics(deployment_uid=deployment_uids[0])

{'end': '2019-01-30T11:14:47.474209Z',
 'metrics': [{'process': 'Accuracy_evaluation_63a87c06-3f04-4174-8bf8-77de76c55193',
   'timestamp': '2019-01-30T11:13:11.803Z',
   'value': {'metrics': [{'name': 'areaUnderROC', 'value': 1.0},
     {'name': 'areaUnderPR', 'value': 1.0}],
    'quality': 1.0,
    'threshold': 0.7}},
  {'process': 'Accuracy_evaluation_87a69dfe-3830-4e47-9372-1c48992ae57c',
   'timestamp': '2019-01-30T11:13:29.674Z',
   'value': {'metrics': [{'name': 'areaUnderROC', 'value': 1.0},
     {'name': 'areaUnderPR', 'value': 1.0}],
    'quality': 1.0,
    'threshold': 0.7}}],
 'start': '2019-01-30T10:07:22.455Z'}

<a id="datamart"></a>
## 6. Get the logged data

### 6.1 Payload logging

#### Print schema of payload_logging table

In [50]:
subscription.payload_logging.print_table_schema()

scoring_id,string,False
scoring_timestamp,timestamp,False
deployment_id,string,False
asset_revision,string,True
EmploymentDuration,string,True
InstallmentPlans,string,True
CreditHistory,string,True
LoanAmount,integer,True
Job,string,True
LoanDuration,integer,True
Housing,string,True


#### Show (preview) the table

In [51]:
subscription.payload_logging.describe_table()

       LoanAmount  LoanDuration  InstallmentPercent  ExistingCreditsCount  \
count        12.0          12.0                12.0                  12.0   
mean       1889.0          31.0                 3.0                   1.0   
std           0.0           0.0                 0.0                   0.0   
min        1889.0          31.0                 3.0                   1.0   
25%        1889.0          31.0                 3.0                   1.0   
50%        1889.0          31.0                 3.0                   1.0   
75%        1889.0          31.0                 3.0                   1.0   
max        1889.0          31.0                 3.0                   1.0   

        Age  Dependents  CurrentResidenceDuration  
count  12.0        12.0                      12.0  
mean   32.0         1.0                       3.0  
std     0.0         0.0                       0.0  
min    32.0         1.0                       3.0  
25%    32.0         1.0                       

#### Return the table content as pandas dataframe

In [52]:
pandas_df = subscription.payload_logging.get_table_content(format='pandas')

### 6.2 Feedback logging

Check the schema of table.

In [53]:
subscription.feedback_logging.print_table_schema()

EmploymentDuration,string,True
InstallmentPlans,string,True
CreditHistory,string,True
LoanAmount,integer,True
Job,string,True
LoanDuration,integer,True
Housing,string,True
InstallmentPercent,integer,True
Telephone,string,True
OwnsProperty,string,True
Sex,string,True


Preview table content.

In [54]:
subscription.feedback_logging.show_table()

greater_7,bank,outstanding_credit,884,skilled,18,own,4,yes,car_other,male,1,none,car_new,yes,less_100,36,0_to_200,2,4,Risk,2019-01-30 11:13:10.699000+00:00
greater_7,bank,outstanding_credit,884,skilled,18,own,4,yes,car_other,male,1,none,car_new,yes,less_100,36,0_to_200,2,4,Risk,2019-01-30 11:13:10.699000+00:00
greater_7,bank,outstanding_credit,884,skilled,18,own,4,yes,car_other,male,1,none,car_new,yes,less_100,36,0_to_200,2,4,Risk,2019-01-30 11:13:10.699000+00:00
greater_7,bank,outstanding_credit,884,skilled,18,own,4,yes,car_other,male,1,none,car_new,yes,less_100,36,0_to_200,2,4,Risk,2019-01-30 11:13:10.699000+00:00
greater_7,bank,outstanding_credit,884,skilled,18,own,4,yes,car_other,male,1,none,car_new,yes,less_100,36,0_to_200,2,4,Risk,2019-01-30 11:13:10.699000+00:00
greater_7,bank,outstanding_credit,884,skilled,18,own,4,yes,car_other,male,1,none,car_new,yes,less_100,36,0_to_200,2,4,Risk,2019-01-30 11:13:10.699000+00:00
greater_7,bank,outstanding_credit,884,skilled,18,own,4,yes,car_other,male,1,none,car_new,yes,less_100,36,0_to_200,2,4,Risk,2019-01-30 11:13:10.699000+00:00
greater_7,bank,outstanding_credit,884,skilled,18,own,4,yes,car_other,male,1,none,car_new,yes,less_100,36,0_to_200,2,4,Risk,2019-01-30 11:13:10.699000+00:00
greater_7,bank,outstanding_credit,884,skilled,18,own,4,yes,car_other,male,1,none,car_new,yes,less_100,36,0_to_200,2,4,Risk,2019-01-30 11:13:10.699000+00:00
greater_7,bank,outstanding_credit,884,skilled,18,own,4,yes,car_other,male,1,none,car_new,yes,less_100,36,0_to_200,2,4,Risk,2019-01-30 11:13:10.699000+00:00


Describe table (calulcate basic statistics).

In [55]:
subscription.feedback_logging.describe_table()

       LoanAmount  LoanDuration  InstallmentPercent  ExistingCreditsCount  \
count        20.0          20.0                20.0                  20.0   
mean        884.0          18.0                 4.0                   1.0   
std           0.0           0.0                 0.0                   0.0   
min         884.0          18.0                 4.0                   1.0   
25%         884.0          18.0                 4.0                   1.0   
50%         884.0          18.0                 4.0                   1.0   
75%         884.0          18.0                 4.0                   1.0   
max         884.0          18.0                 4.0                   1.0   

        Age  Dependents  CurrentResidenceDuration  
count  20.0        20.0                      20.0  
mean   36.0         2.0                       4.0  
std     0.0         0.0                       0.0  
min    36.0         2.0                       4.0  
25%    36.0         2.0                       

Get table content.

In [56]:
feedback_pd = subscription.feedback_logging.get_table_content(format='pandas')

### 6.3 Quality metrics table

In [57]:
subscription.quality_monitoring.print_table_schema()

ts,timestamp,False
quality,float,False
quality_threshold,float,False
binding_id,string,False
subscription_id,string,False
deployment_id,string,True
process,string,False
asset_revision,string,True


In [58]:
subscription.quality_monitoring.show_table()

2019-01-30 11:13:11.803000+00:00,1.0,0.7,dd096f50-15eb-4d22-bc5f-39aff54acc98,085460ef94636166aea5800e9ea26168,563f01d37f720857b95c557dc76176ad,Accuracy_evaluation_63a87c06-3f04-4174-8bf8-77de76c55193,
2019-01-30 11:13:29.674000+00:00,1.0,0.7,dd096f50-15eb-4d22-bc5f-39aff54acc98,085460ef94636166aea5800e9ea26168,563f01d37f720857b95c557dc76176ad,Accuracy_evaluation_87a69dfe-3830-4e47-9372-1c48992ae57c,


### 6.4 Performance metrics table

In [59]:
subscription.performance_monitoring.print_table_schema()

ts,timestamp,False
scoring_time,float,False
scoring_records,object,False
binding_id,string,False
subscription_id,string,False
deployment_id,string,True
process,string,False
asset_revision,string,True


In [60]:
subscription.performance_monitoring.show_table()

2019-01-30 11:09:17.233054+00:00,0.0,1,dd096f50-15eb-4d22-bc5f-39aff54acc98,085460ef94636166aea5800e9ea26168,563f01d37f720857b95c557dc76176ad,,
2019-01-30 11:09:17.233160+00:00,0.0,1,dd096f50-15eb-4d22-bc5f-39aff54acc98,085460ef94636166aea5800e9ea26168,563f01d37f720857b95c557dc76176ad,,
2019-01-30 11:09:17.233073+00:00,0.0,1,dd096f50-15eb-4d22-bc5f-39aff54acc98,085460ef94636166aea5800e9ea26168,563f01d37f720857b95c557dc76176ad,,
2019-01-30 11:09:17.233034+00:00,0.0,1,dd096f50-15eb-4d22-bc5f-39aff54acc98,085460ef94636166aea5800e9ea26168,563f01d37f720857b95c557dc76176ad,,
2019-01-30 11:09:17.233143+00:00,0.0,1,dd096f50-15eb-4d22-bc5f-39aff54acc98,085460ef94636166aea5800e9ea26168,563f01d37f720857b95c557dc76176ad,,
2019-01-30 11:09:17.232986+00:00,0.0,1,dd096f50-15eb-4d22-bc5f-39aff54acc98,085460ef94636166aea5800e9ea26168,563f01d37f720857b95c557dc76176ad,,
2019-01-30 11:09:17.233108+00:00,0.0,1,dd096f50-15eb-4d22-bc5f-39aff54acc98,085460ef94636166aea5800e9ea26168,563f01d37f720857b95c557dc76176ad,,
2019-01-30 11:09:17.233013+00:00,0.0,1,dd096f50-15eb-4d22-bc5f-39aff54acc98,085460ef94636166aea5800e9ea26168,563f01d37f720857b95c557dc76176ad,,
2019-01-30 11:09:17.233125+00:00,0.0,1,dd096f50-15eb-4d22-bc5f-39aff54acc98,085460ef94636166aea5800e9ea26168,563f01d37f720857b95c557dc76176ad,,
2019-01-30 11:09:17.232921+00:00,0.0,1,dd096f50-15eb-4d22-bc5f-39aff54acc98,085460ef94636166aea5800e9ea26168,563f01d37f720857b95c557dc76176ad,,


### 6.5 Data Mart measurement facts table

In [61]:
client.data_mart.get_deployment_metrics()

{'deployment_metrics': [{'asset': {'asset_id': '085460ef94636166aea5800e9ea26168',
    'asset_type': 'model',
    'created_at': '2019-01-09T10:42:59.7933412Z',
    'name': 'GermanCreditRisk.2019.1.9.10.41.58.611',
    'url': 'https://ussouthcentral.services.azureml.net/subscriptions/744bca722299451cb682ed6fb75fb671/services/e13d36b1c48f4080a49e5ae675d816ec/swagger.json'},
   'deployment': {'created_at': '2019-01-09T10:42:59.7933412Z',
    'deployment_id': '563f01d37f720857b95c557dc76176ad',
    'deployment_rn': '/subscriptions/744bca72-2299-451c-b682-ed6fb75fb671/resourceGroups/ai-ops-squad/providers/Microsoft.MachineLearning/webServices/GermanCreditRisk.2019.1.9.10.41.58.611',
    'deployment_type': 'online',
    'name': 'GermanCreditRisk.2019.1.9.10.41.58.611',
    'scoring_endpoint': {'credentials': {'token': 'X/YNeesRdD95FDyXoBm7wvP2LlHD4pZGlMARZv6rh8AfV0Ol0Zb1KiftUJiNmisml7NmhURSpzeGn2UsKzcqZw=='},
     'request_headers': {'Content-Type': 'application/json'},
     'url': 'https://

---

### Authors
Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.